# ASPIICS L2 Batch processing

In [ ]:
from pathlib import Path
import subprocess
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm

def process_l2_image(file, output_dir):
    # print(f"Processing L2 image: {file}")
    cmd = ['python','l2_processor/l2_master.py', str(file), '--cal', 'l2_config.json', '--banding_correction', '--outdir', str(output_dir)]
    result = subprocess.run(cmd, check=True, shell=True)
    return result

## Adjust your directories below:

In [ ]:
# ***************** Input images *******************
input_files = list(Path('PATH/TO/YOUR/L1/FILES/').rglob('aspiics_wb_l1*.fits'))

output_dir = Path('PATH/TO/YOUR/OUTPUT/L2/DIRECTORY/')

# Number of threads for parallel processing
threads = 8

In [ ]:
output_dir.mkdir(parents=True, exist_ok=True)
input_files.sort()

print(f"Found {len(input_files)} L1 images to process.")

# process images in parallel using threads:
with ThreadPoolExecutor(max_workers=threads) as executor:
    futures = {executor.submit(process_l2_image, file, output_dir): file for file in input_files}
    
    with tqdm(total=len(futures), desc="Processing images") as pbar:
        for future in as_completed(futures):
            try:
                future.result()  # to catch exceptions if any
            except Exception as e:
                print(f"Error processing {futures[future]}: {e}")
            pbar.update(1)

